In [26]:
import pandas as pd

df = pd.read_csv("magnetism.csv")
df

,Timestamp,Elapsed time (minutes),Set B field (T),B field (T),Ramp speed (T/min),Voltage (V),Inner Lower Temperature,Outer Lower Temperature,Inner Upper Temperature,Outer Upper Temperature,1st Stage Temperature,Shield Temperature,2nd Stage Temperature,Magnetism_ID
0,02/08/2019 03:27:38 PM,2.805300,0.0,0.00000,0.000000,0.0,281.797983,285.092215,280.715422,283.969736,145.612162,279.920378,204.958080,./T10/20190208_SMS.txt
1,02/08/2019 03:28:01 PM,3.175539,0.0,0.00000,0.000000,0.0,281.768245,285.065814,280.686698,283.941646,145.521580,279.884569,204.900535,./T10/20190208_SMS.txt
2,02/08/2019 03:28:23 PM,3.554124,0.0,0.00000,0.000000,0.0,281.737322,285.041573,280.652349,283.915624,145.428632,279.849350,204.839601,./T10/20190208_SMS.txt
3,02/08/2019 03:28:46 PM,3.933961,0.0,0.00000,0.000000,0.0,281.704371,285.016645,280.621349,283.886878,145.337971,279.809776,204.778354,./T10/20190208_SMS.txt
4,02/08/2019 03:29:09 PM,4.314683,0.0,0.00000,0.000000,0.0,281.673613,284.987751,280.583256,283.856319,145.245554,279.771382,204.720981,./T10/20190208_SMS.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2789982,03/20/2019 10:24:29 AM,9833.385706,0.0,-0.00000,0.006867,0.0,5.073398,5.019095,4.877788,4.965012,41.333217,63.687044,3.241164,././T10/20190313_SMS.txt
2789983,03/20/2019 10:24:35 AM,9833.491933,0.0,-0.00426,0.006867,0.0,5.071438,5.017297,4.876068,4.963015,41.332443,63.700309,3.242243,././T10/20190313_SMS.txt
2789984,03/20/2019 10:24:42 AM,9833.604840,0.0,-0.00000,0.006867,0.0,5.069822,5.015468,4.874303,4.960119,41.331908,63.710247,3.248909,././T10/20190313_SMS.txt
2789985,03/20/2019 10:24:48 AM,9833.707356,0.0,-0.00000,0.006867,0.0,5.068294,5.013746,4.872636,4.958283,41.331908,63.720780,3.325273,././T10/20190313_SMS.txt


In [27]:
float64_cols = list(df.select_dtypes(include='float64'))
df[float64_cols] = df[float64_cols].astype('float32')

In [28]:
import pandas as pd
from darts import TimeSeries

# Assuming 'your_dataset.csv' is a string containing the dataset

# df = pd.read_csv(pd.compat.StringIO('your_dataset.csv'))

# Convert the 'Timestamp' column to datetime type
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

time_series_list = []
for ccr_id, group in df.groupby('Magnetism_ID'):
    # Sort the group by 'Timestamp' if not already sorted
    group.sort_values('Timestamp', inplace=True)
    
    # Set the 'Timestamp' column as the index
    group.set_index('Timestamp', inplace=True)
    
    # Exclude the 'CCR_ID' column from numeric conversion and resampling
    value_cols = ['Elapsed time (minutes)', 'Set B field (T)', 'B field (T)', 'Ramp speed (T/min)', 'Voltage (V)', 'Inner Lower Temperature', 'Outer Lower Temperature', 'Inner Upper Temperature', 'Outer Upper Temperature', '1st Stage Temperature', 'Shield Temperature', '2nd Stage Temperature', ]
    # value_cols = ['Elapsed time (minutes)','Setpoint (K)','Sensor A (K)','Sensor B (K)', 'Heater Output % (0-100)', 'Heater Range (integer 0-5)', 'Slope','Intercept','Sensor C (K) or sensor A (sensor unit)','Sensor D (K) or sensor B (sensor unit)']
    
    # Convert the numeric columns to numeric format, handling non-numeric values as NaN
    for col in value_cols:
        group[col] = pd.to_numeric(group[col], errors='coerce')
    
    # Select only numeric columns for resampling
    numeric_cols = group.select_dtypes(include='number').columns
    
    # Resample the numeric columns to a fixed frequency
    group_resampled = group[numeric_cols].resample('1T').mean()  # Adjust '5S' to the desired frequency
    
    # Handle missing values if necessary
    group_resampled = group_resampled.fillna(method='ffill')  # Forward fill missing values, or choose another appropriate method
    
    # Convert the resampled DataFrame to a Darts TimeSeries object
    time_series = TimeSeries.from_dataframe(group_resampled, value_cols=value_cols)
    
    time_series_list.append(time_series)


/tmp/ipykernel_2996322/2093529370.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Timestamp'] = pd.to_datetime(df['Timestamp'])


In [ ]:
print(len(time_series_list))

In [29]:
updated_list = []
for time_series in time_series_list:
    for i in range(len(time_series)//1000):
        updated_list.append(time_series[i*1000:(i+1)*1000])
print(len(updated_list))

307


In [ ]:
updated_list[1].columns

In [ ]:
print(len(time_series_list[1]))

In [32]:
import pandas as pd
from darts import TimeSeries

# Assuming 'your_dataset.csv' is a string containing the dataset
# df = pd.read_csv(pd.compat.StringIO('your_dataset.csv'))

# Convert the 'Timestamp' column to datetime type
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Create a deep copy of the DataFrame to preserve the original 'df'
copy = df.copy()

subsection = 0
for magnetism_id, group in df.groupby('Magnetism_ID'):
    # Sort the group by 'Timestamp' if not already sorted
    group.sort_values('Timestamp', inplace=True)
    
    # Create a new 'Subsection' column and initialize it with NaN values in the 'copy' DataFrame
    copy.loc[group.index, 'Subsection'] = float('nan')
    
    # Create subsections of 600 records each
    num_subsections = len(group) // 1000
    for i in range(num_subsections):
        copy.loc[group.index[i * 1000: (i + 1) * 1000], 'Subsection'] = subsection
        subsection += 1

    # Assign the remaining records to the last subsection
    remainder = len(group) % 1000
    # if remainder > 1800:
    #     copy.loc[group.index[-remainder:], 'Subsection'] = subsection
    #     subsection += 1

# If you want to see the 'Subsection' column in the 'copy' DataFrame:
# print(copy['Subsection'])


In [96]:
print(len(copy))

2789987


In [97]:
2789987//2000

1394

In [33]:
copy.to_csv('magnetism_split_1000.csv', index=False)

In [87]:
result = copy[copy['Subsection'] == 0]

In [88]:
print(result)

                  Timestamp  Elapsed time (minutes)  Set B field (T)  \
1975971 2019-02-19 12:55:40                0.195535              0.0   
1975972 2019-02-19 12:55:46                0.309618              0.0   
1975973 2019-02-19 12:55:53                0.417890              0.0   
1975974 2019-02-19 12:55:59                0.524773              0.0   
1975975 2019-02-19 12:56:06                0.630357              0.0   
...                     ...                     ...              ...   
1977966 2019-02-19 16:32:16              216.798080              0.0   
1977967 2019-02-19 16:32:22              216.908188              0.0   
1977968 2019-02-19 16:32:29              217.019394              0.0   
1977969 2019-02-19 16:32:36              217.129593              0.0   
1977970 2019-02-19 16:32:42              217.238525              0.0   

         B field (T)  Ramp speed (T/min)  Voltage (V)  \
1975971      0.00426            0.091565         -0.1   
1975972      0.00000 

In [30]:
import pickle
with open('timeseries_1000','wb') as f: pickle.dump(updated_list, f)

In [11]:
cd ..

'/storage/rwp2/Pipeline/Preprocessing'